In [178]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from requests.utils import unquote
import re

In [179]:
def get_kapt_energyconsumption(kaptcode, date):
    service_key = "iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D"
    
    url = "http://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService"  # 엔드포인트 확인 필요

    params = {
        "serviceKey": service_key,
        "kaptCode": kaptcode,
        "reqDate": date,
        "_type": "xml"
    }

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }

    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        
        # 응답 출력하여 확인
        print(response.text)

        data_list = []
        for item in root.findall(".//item"):
            data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
            data_list.append(data_dict)
        
        df = pd.DataFrame(data_list)
        return df
    else:
        print(f"오류 발생: HTTP {response.status_code}")
        return None

# 테스트
df = get_kapt_energyconsumption("A10027953", "202201")
print(df)


오류 발생: HTTP 500
None


In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
from urllib.parse import unquote

def get_kapt_energyconsumption(kaptcode, date):
    """단일 아파트 코드(kaptcode)와 날짜(date)에 대해 에너지 소비량을 조회"""
    
    # 공공데이터포털 서비스 키 (디코딩)
    service_key = unquote("iCNxo2r0TdZnnV63%2FItO%2BQrOUqJakXCxx%2Fm20BsCp53DGZzJMDd1%2F7jOGLYQE%2BSn%2B1EQeSeIhUsTIyQ5dYgy4Q%3D%3D")
    
    # API 엔드포인트
    url = "https://apis.data.go.kr/1611000/ApHusEnergyUseInfoOfferService/getHsmpApHusUsgQtyInfoSearch"

    # 요청 파라미터 설정
    params = {
        "serviceKey": service_key,  
        "kaptCode": kaptcode,       
        "reqDate": date,            
        "_type": "xml"              
    }

    # 요청 헤더 설정
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36",
        "Accept": "application/xml"
    }

    # API 요청
    response = requests.get(url, params=params, headers=headers)

    # 응답 상태 코드 확인
    if response.status_code == 200:
        root = ET.fromstring(response.text)
        data_list = []

        # XML에서 item 태그 찾기
        for item in root.findall(".//item"):
            data_dict = {child.tag: child.text if child.text else 'N/A' for child in item}
            data_list.append(data_dict)

        # 데이터가 없는 경우 확인 메시지 출력
        if not data_list:
            print(f"데이터 없음: kaptCode={kaptcode}, date={date}")

        # 데이터프레임 변환
        return pd.DataFrame(data_list)

    else:
        print(f"오류 발생: HTTP {response.status_code} (kaptCode={kaptcode}, date={date})")
        print("응답 내용:", response.text)  
        return None

def get_multiple_kapt_energyconsumption(kaptcodes, dates):
    """여러 개의 아파트 코드(kaptcodes)와 날짜(dates)에 대해 에너지 소비량을 조회"""
    
    all_results = []  # 모든 데이터를 저장할 리스트

    for kaptcode in kaptcodes:
        for date in dates:
            df = get_kapt_energyconsumption(kaptcode, date)
            if df is not None and not df.empty:
                df["kaptCode"] = kaptcode  # 아파트 코드 추가
                df["reqDate"] = date  # 요청 날짜 추가
                all_results.append(df)

    # 모든 데이터를 하나의 데이터프레임으로 합치기
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        final_df.rename(columns = {
                "elect" : "전기사용금액",
                "gas" : "가스사용금액",
                "heat" : "난방사용금액",
                "helect" : "전기사용량",
                "hgas" : "가스사용량",
                "hheat" : "난방사용량",
                "hwaterCool" : "냉수사용량",
                "hwaterHot" : "온수사용량",
                # "kaptCode" : "아파트코드",
                "waterCool" : "냉수사용금액",
                "waterHot" : "온수사용금액",
                "reqDate" : "집계날짜"}, inplace=True)
        return final_df
    
    else:
        print("❗ 모든 요청에서 데이터를 가져오지 못했습니다.")
        return pd.DataFrame()

# 테스트 실행 
df = pd.read_csv("/Users/ijongseung/house_electric/address_data/seoul_data.csv") # 

kaptcodes = df["kaptCode"].tolist()  # 여러 개의 아파트 코드
dates = ["202401"]  # 일단 가장 최근 날짜로 변형한다.

energy_data = get_multiple_kapt_energyconsumption(kaptcodes, dates)

In [15]:
df = pd.read_csv("/Users/ijongseung/house_electric/address_data/seoul_data.csv") # 3261

In [16]:
energy_data # 열 변경 필요

# 이후 결합 실시
df = pd.merge(df, energy_data, on="kaptCode", how="outer")
df.rename(columns = {"doroJuso": "address"}, inplace=True)

In [172]:
# 송파구 데이터만 수집
df_gangnam = df[df["kaptAddr"].str.contains("강남구")]
# - 을 제거-
df_gangnam["kaptAddr1"] = df_gangnam["kaptAddr"].str.replace("-", "", regex=True).str.strip()

df_gangnam.rename(columns = {"doroJuso": "address"}, inplace=True)

/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_69659/3403312328.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["kaptAddr1"] = df_gangnam["kaptAddr"].str.replace("-", "", regex=True).str.strip()
/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_69659/3403312328.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam.rename(columns = {"doroJuso": "address"}, inplace=True)


In [177]:
df_gangnam[["address", "kaptAddr"]].to_csv("gangnam.csv")

In [175]:
df_gangnam = 
import re
import requests
import pandas as pd

# 🔹 Kakao API Key 입력
KAKAO_API_KEY = "b83a0138ed0d17b68f0c86fe6808b74d"
YOUR_KAKAO_JS_API_KEY = "41c870c490aebd3810bd1152b41c792e"

def clean_address(address):
    if pd.isna(address):  # NaN 값이 들어오면 그대로 반환
        return address
    return re.sub(r"(\d+-?\d*)\s?.*", r"\1", address).strip()


# 🔹 지번주소 → 도로명주소 변환 함수
def get_road_address(jibun_address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    params = {"query": jibun_address}

    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            road_address = result["documents"][0].get("road_address", {}).get("address_name")
            if road_address:
                print(f"✅ 도로명 주소 변환 성공: {jibun_address} → {road_address}")
                return road_address
            else:
                print(f"❌ 도로명 주소 변환 실패: {jibun_address} (원본 유지)")
                return jibun_address  # 변환 실패 시 원본 유지
    else:
        print(f"❌ API 요청 실패: {response.status_code}, {response.text}")

    return jibun_address  # 변환 실패 시 원본 유지

# 🔹 주소를 위도, 경도로 변환하는 함수
def get_lat_lon(address):
    url = "https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}
    
    params = {"query": address}
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        result = response.json()
        if result["documents"]:
            lat = float(result["documents"][0]["y"])
            lon = float(result["documents"][0]["x"])
            return lat, lon
        else:
            print(f"❌ 주소 변환 실패: {address}")  
    else:
        print(f"API 요청 실패: {response.status_code}, {response.text}")

    return None, None


# 🔹 주소 클리닝 (숫자 뒤의 아파트 이름 제거, 지번 유지)
df_gangnam["cleaned_address"] = df_gangnam["kaptAddr"].apply(clean_address)

# 🔹 기존에 "address" 열이 존재하는 경우 변환 스킵
if "address" not in df_gangnam.columns:
    df_gangnam["address"] = None  # "address" 열이 없으면 생성

# 🔹 address 열이 None이면 변환 실행, 값이 있으면 기존 값 유지
df_gangnam["address"] = df_gangnam.apply(
    lambda row: get_road_address(row["cleaned_address"]) if pd.isna(row["address"]) else row["address"], axis=1
)

# 🔹 도로명주소 기반으로 위도/경도 변환
df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)

# 🔹 변환되지 않은 데이터 제거
df_gangnam = df_gangnam.dropna(subset=["lat", "lon"])

# 🔹 전기사용량 열 추가 (전기사용량이 없을 경우 0으로 대체)
if "전기사용량" not in df_gangnam.columns:
    df_gangnam["전기사용량"] = 0

# 🔹 HTML 파일 생성
html_template = f"""
<!DOCTYPE html>
<html>
<head>
    <meta charset="utf-8"/>
    <title>Kakao 지도 - 송파구 아파트</title>
    <script type="text/javascript" src="//dapi.kakao.com/v2/maps/sdk.js?appkey=41c870c490aebd3810bd1152b41c792e"></script>
</head>
<body>
    <div id="map" style="width:800px;height:600px;"></div>
    <script>
        var map = new kakao.maps.Map(document.getElementById('map'), {{
            center: new kakao.maps.LatLng({df_gangnam['lat'].mean()}, {df_gangnam['lon'].mean()}), 
            level: 6 
        }});

        var positions = {df_gangnam.to_json(orient='records')};

        positions.forEach(function(position) {{
            var marker = new kakao.maps.Marker({{
                map: map,
                position: new kakao.maps.LatLng(position.lat, position.lon)
            }});
            
            var infowindow = new kakao.maps.InfoWindow({{
                content: '<div style="padding:5px;"><b>주소:</b> ' + position.address + '<br>' +
                         '<b>전기사용량:</b> ' + position.전기사용량 + ' kWh</div>'
            }});

            kakao.maps.event.addListener(marker, 'click', function() {{
                infowindow.open(map, marker);
            }});
        }});
    </script>
</body>
</html>
"""

# 🔹 HTML 파일 저장
with open("map.html", "w", encoding="utf-8") as f:
    f.write(html_template)

print("HTML 파일이 생성되었습니다. 'python3 -m http.server 8080'을 실행하고 http://localhost:8080/map.html 에 접속하세요!")

/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_69659/1701710651.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["cleaned_address"] = df_gangnam["kaptAddr"].apply(clean_address)


✅ 도로명 주소 변환 성공: 서울특별시 강남구 논현동 196- → 서울 강남구 강남대로112길 41
✅ 도로명 주소 변환 성공: 서울특별시 강남구 논현동 241-1 → 서울 강남구 학동로 338
✅ 도로명 주소 변환 성공: 서울특별시 강남구 청담동 43- → 서울 강남구 학동로 409
✅ 도로명 주소 변환 성공: 서울특별시 강남구 신사동 506-9 → 서울 강남구 압구정로2길 20
✅ 도로명 주소 변환 성공: 서울특별시 강남구 삼성동 3-3 → 서울 강남구 학동로 412
✅ 도로명 주소 변환 성공: 서울특별시 강남구 논현동 107-44 → 서울 강남구 언주로136길 21
✅ 도로명 주소 변환 성공: 서울특별시 강남구 논현동 195-1 → 서울 강남구 봉은사로11길 12
✅ 도로명 주소 변환 성공: 서울특별시 강남구 삼성동 126- → 서울 강남구 삼성로99길 14
✅ 도로명 주소 변환 성공: 서울특별시 강남구 삼성동 104- → 서울 강남구 봉은사로105길 12-7
✅ 도로명 주소 변환 성공: 서울특별시 강남구 역삼동 761-10 → 서울 강남구 언주로 316
✅ 도로명 주소 변환 성공: 서울특별시 강남구 청담동 67-1 → 서울 강남구 학동로97길 51
✅ 도로명 주소 변환 성공: 서울특별시 강남구 논현동 40- → 서울 강남구 논현로131길 44


/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_69659/1701710651.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam["address"] = df_gangnam.apply(


HTML 파일이 생성되었습니다. 'python3 -m http.server 8080'을 실행하고 http://localhost:8080/map.html 에 접속하세요!


/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_69659/1701710651.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)
/var/folders/7n/zyzjp7wj0cn68mx_s16mw1m00000gn/T/ipykernel_69659/1701710651.py:73: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_gangnam[["lat", "lon"]] = df_gangnam["address"].apply(get_lat_lon).apply(pd.Series)


In [162]:
df_gangnam["kaptAddr"].str.contains("서울특별시 강남구 개포동 138-")

389     False
390     False
391     False
392     False
393     False
        ...  
3102    False
3152    False
3165    False
3170    False
3223    False
Name: kaptAddr, Length: 204, dtype: bool

In [167]:
df_gangnam[df_gangnam["kaptAddr"].str.contains("역삼동")]


,bjdCode,codeAptNm,codeHallNm,codeHeatNm,codeMgrNm,codeSaleNm,doroJuso,hoCnt,kaptAcompany,kaptAddr,...,kaptdaCnt,ktownFlrNo,privArea,zipcode,kaptAddr1,cleaned_address,address,lat,lon,전기사용량
392,1168010100,주상복합,계단식,개별난방,위탁관리,분양,서울특별시 강남구 언주로85길 13,164.0,역삼 주상복합조합,서울특별시 강남구 역삼동 722- 역삼경남,...,164.0,17.0,13661.4400,6222.0,서울특별시 강남구 역삼동 722 역삼경남,서울특별시 강남구 역삼동 722-,서울 강남구 언주로85길 13,37.500567,127.042382,0
393,1168010100,아파트,계단식,중앙난방,위탁관리,분양,서울특별시 강남구 선릉로 423,206.0,선릉123지여구택조합,서울특별시 강남구 역삼동 716- 역삼동부센트레빌,...,206.0,21.0,14922.5200,6213.0,서울특별시 강남구 역삼동 716 역삼동부센트레빌,서울특별시 강남구 역삼동 716-,서울 강남구 선릉로 423,37.502503,127.049442,0
394,1168010100,아파트,복도식,개별난방,위탁관리,분양,서울특별시 강남구 도곡로3길 26,216.0,한일합섬,서울특별시 강남구 역삼동 836- 역삼한스빌아파트,...,216.0,12.0,8575.5900,6254.0,서울특별시 강남구 역삼동 836 역삼한스빌아파트,서울특별시 강남구 역삼동 836-,서울 강남구 도곡로3길 26,37.491983,127.032956,0
395,1168010100,아파트,계단식,지역난방,위탁관리,분양,서울특별시 강남구 역삼로 309,288.0,삼성물산,서울특별시 강남구 역삼동 713-7 역삼래미안펜타빌,...,288.0,24.0,42490.3000,6215.0,서울특별시 강남구 역삼동 7137 역삼래미안펜타빌,서울특별시 강남구 역삼동 713-7,서울 강남구 역삼로 309,37.500299,127.046905,0
396,1168010100,아파트,혼합식,지역난방,위탁관리,분양,서울특별시 강남구 역삼로 307,541.0,개나리2차저층재건축조합,서울특별시 강남구 역삼동 713-11 역삼아이파크,...,541.0,21.0,52062.9590,6215.0,서울특별시 강남구 역삼동 71311 역삼아이파크,서울특별시 강남구 역삼동 713-11,서울 강남구 역삼로 307,37.499049,127.044876,0
397,1168010100,주상복합,혼합식,개별난방,위탁관리,분양,서울특별시 강남구 테헤란로4길 46,166.0,한국자산신탁(주),서울특별시 강남구 역삼동 826-37 역삼쌍용플래티넘밸류,...,166.0,15.0,15705.9400,6241.0,서울특별시 강남구 역삼동 82637 역삼쌍용플래티넘밸류,서울특별시 강남구 역삼동 826-37,서울 강남구 테헤란로4길 46,37.495548,127.030065,0
513,1168010100,주상복합,복도식,지역난방,위탁관리,분양,서울특별시 강남구 언주로 427,457.0,대우건설,서울특별시 강남구 역삼동 720-25 대우디오빌역삼,...,457.0,26.0,18737.9830,6221.0,서울특별시 강남구 역삼동 72025 대우디오빌역삼,서울특별시 강남구 역삼동 720-25,서울 강남구 언주로 427,37.501476,127.042690,0
514,1168010100,아파트,계단식,지역난방,위탁관리,분양,서울특별시 강남구 선릉로69길 20,840.0,(주)대림산업,서울특별시 강남구 역삼동 755-4 역삼e-편한세상,...,840.0,25.0,58385.7550,6217.0,서울특별시 강남구 역삼동 7554 역삼e편한세상,서울특별시 강남구 역삼동 755-4,서울 강남구 선릉로69길 20,37.498747,127.050924,0
515,1168010100,아파트,계단식,지역난방,위탁관리,분양,서울특별시 강남구 선릉로69길 19,1050.0,영동1재건축조합,서울특별시 강남구 역삼동 757- 역삼래미안,...,1050.0,22.0,69465.8600,6218.0,서울특별시 강남구 역삼동 757 역삼래미안,서울특별시 강남구 역삼동 757-,서울 강남구 선릉로69길 19,37.498126,127.050691,0
595,1168010100,아파트,계단식,지역난방,위탁관리,분양,서울특별시 강남구 역삼로 314,332.0,대우건설,서울특별시 강남구 역삼동 755-1 역삼개나리푸르지오,...,332.0,24.0,35538.3408,6217.0,서울특별시 강남구 역삼동 7551 역삼개나리푸르지오,서울특별시 강남구 역삼동 755-1,서울 강남구 역삼로 314,37.499615,127.049303,0


### 열 처리 방법

###
df_songpa.rename(columns={
    'kaptName': '단지명',
    'kaptAddr': '법정동주소',
    'codeSaleNm': '분양형태',
    'codeHeatNm': '난방방식',
    'kaptTarea': '건축물대장상 연면적',
    'kaptDongCnt': '동수',
    'hoCnt': '세대수',
    'kaptAcompany': '시공사',
    'kaptBcompany': '시행사',
    'kaptTel': '관리사무소연락처',
    'kaptFax': '관리사무소팩스',
    'kaptUrl': '홈페이지주소',
    'codeAptNm': '단지분류',
    'address': '도로명주소',
    'kaptdaCnt': '호수',
    'codeMgrNm': '관리방식',
    'codeHallNm': '복도유형',
    'kaptUsedate': '사용승인일',
    'kaptMarea': '관리비부과면적',
    'kaptMparea_60': '전용면적별 세대현황_60㎡이하',
    'kaptMparea_85': '전용면적별 세대현황_85㎡이하',
    'kaptMparea_135': '전용면적별 세대현황_135㎡이상',
    'kaptMparea_136': '전용면적별 세대현황_136㎡이상',
    'privArea': '단지 전용면적합',
    'bjdCode': '법정동코드'
}, inplace=True)